In [2]:
import pandas as pd
import numpy as np
#import tabula
import math
import os
from datetime import date

In [5]:
class VacsTable:
    """
    VacsTable class defines a vacants table and methods to clean that table
    """
    table_types = {'vacants': [        #list of vacants tables admitted. 
                            'LD-PA common',   #ld-pa vacants usually generate (2, 14) DFs
                            'CM common',      #ld-pa vacants usually generate (2, 15) DFs
                            '2-16 shape', 
                            np.nan],
                  'posts': [np.nan]
                 }
    
    def __init__(self, df):
        self.data = df
        self.type = 'unknown'   #property for type of table
        self.vac_qty = 0        #property for number of vacants in the table
        self.__get_type()         #call get_type() method at instantiation moment
        self.__get_vac_qty()      #call get_vac_qty() method at instantiation moment

    def __get_type(self):      #method to determine the type of vacants table at hand
        if df.shape == (2, 14):
            self.type = 'LD-PA common'        #ld-pa vacants usually generate (2, 14) DFs
            assert self.type in self.types
            
        elif df.shape == (2, 15):
            self.type = 'CM common'           #CM vacants usually generate (2, 15) DFs
            assert self.type in self.types

        elif df.shape == (2, 16):
            self.type = '2-16 common'        #xxx vacants usually generate (2, 16) DFs
            assert self.type in self.types
            
        else:
            pass
        
    def __get_vac_qty(self):       #method to extract number of vacants in the table
        pass
        
    def extract_vacs(self):        #method to extract vacants. Calls the appropriate private method
        if self.type == t1:
            self.__extract_t1()

        elif self.type == t2:
            self.__extract_t2()
            
        else: pass
    
    def __extract_t1(self):        #method to extract vacants in the type 1 table
        pass
    
    def __extract_t2(self):        #method to extract vacants in the type x table
        pass
    
    def __extract_tx(self):        #method to extract vacants in the type x table
        pass

In [ ]:
for count, df in enumerate(read_pdf):
    table = VacsTable(df)
    if table.type in table_types['vacants']:
        table.extract_vacs()
        
    elif table.type in table_types['posts']:
        table.extract_posts()
        
    else: pass